# DuckDB and DyalogAPL with Vega-Lite

In [5]:
⎕pw←32767 ⋄ ⎕PP←34

In [6]:
]link.create # ../dyalog-jupyter-vegalite

Linked: # ←→ /home/kimmo/dyalog-jupyter-vegalite/../dyalog-jupyter-vegalite

In [7]:
duckdb.init 'lib/'
_db←duckdb.open ':memory:'
_con←duckdb.connect _db
_con

94166708459728

In [8]:
duckdb.query _con 'CREATE TABLE elec AS SELECT * FROM "../dyalog-duckdb-parquet/electricity.parquet"' ⍝ data/electricity.parquet

┌───────┬────┬─┐
│┌─────┐│5878│5│
││Count││ │ │
│└─────┘│ │ │
└───────┴────┴─┘

In [9]:
duckdb.toTable duckdb.query _con 'PRAGMA table_info(elec)'

┌───┬───────────┬─────────┬───────┬──────────┬──┐
│cid│name │type │notnull│dflt_value│pk│
├───┼───────────┼─────────┼───────┼──────────┼──┤
│0 │time │TIMESTAMP│0 │ [Null] │0 │
├───┼───────────┼─────────┼───────┼──────────┼──┤
│1 │consumption│DOUBLE │0 │ [Null] │0 │
├───┼───────────┼─────────┼───────┼──────────┼──┤
│2 │price │DOUBLE │0 │ [Null] │0 │
└───┴───────────┴─────────┴───────┴──────────┴──┘

In [10]:
duckdb.toTable duckdb.query _con 'SELECT * FROM elec ORDER BY time LIMIT 5'

┌───────────────────┬───────────┬───────┐
│time │consumption│price │
├───────────────────┼───────────┼───────┤
│2021 12 31 23 0 0 0│5.71 │5.7784 │
├───────────────────┼───────────┼───────┤
│2022 1 1 0 0 0 0 │9.32 │5.12492│
├───────────────────┼───────────┼───────┤
│2022 1 1 1 0 0 0 │8.01 │5.23032│
├───────────────────┼───────────┼───────┤
│2022 1 1 2 0 0 0 │4.93 │5.50188│
├───────────────────┼───────────┼───────┤
│2022 1 1 3 0 0 0 │4 │4.67108│
└───────────────────┴───────────┴───────┘

In [14]:
]multiline
duckdb.toTable duckdb.query _con ```
SELECT 
    year(time) as year,
    month(time) as month,
    sum(consumption) as consumption,
    sum(price*consumption/100) as costs,
    avg(price) as avg_price,
    sum(price*consumption)/sum(consumption) as weighted_price 
FROM elec
WHERE year(time)>2021 
GROUP BY year(time),month(time)
```

┌────┬─────┬───────────┬───────────┬───────────┬──────────────┐
│year│month│consumption│costs │avg_price │weighted_price│
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│1 │3906.38 │511.4491408│13.24715667│13.09266228 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│2 │3078.02 │306.7896531│10.0258244 │9.967110452 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│3 │2777.89 │286.8994659│10.74263833│10.32796352 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│4 │2145.95 │195.0728148│9.8475375 │9.090277721 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│5 │1754.37 │254.6228395│16.411175 │14.51363393 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│6 │1456.76 │216.1720991│17.40090278│14.83923907 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│7 │1523.36 │319.4839924│22.80603833│20.97232384 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│8 │1349.76 │409.8211578│32.43009333│30.36252058 │
├────┼─────┼───────────┼───────────┼───────────┼──────────────┤
│2022│9 │67.63 │24.74610811│40.62526578│36.59043045 │
└────┴─────┴───────────┴───────────┴───────────┴──────────────┘

In [15]:
duckdb.query _con 'SELECT strftime(time,''%Y-%m-%dT%H:%MZ'') as date,price as value FROM elec LIMIT 5'

┌────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────┐
│┌────┬─────┐│┌───────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┐│17 11│
││date│value│││┌─────────────────┬─────────────────┬─────────────────┬─────────────────┬─────────────────┐│5.7784 5.12492 5.23032 5.50188 4.67108││ │
│└────┴─────┘│││2021-12-31T23:00Z│2022-01-01T00:00Z│2022-01-01T01:00Z│2022-01-01T02:00Z│2022-01-01T03:00Z││ ││ │
│ ││└─────────────────┴─────────────────┴─────────────────┴─────────────────┴─────────────────┘│ ││ │
│ │└───────────────────────────────────────────────────────────────────────────────────────────┴──────────────────────────────────────┘│ │
└────────────┴────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┴─────┘

```apl
json←toJson in
col data←2↑in
data←{a←⍵ ⋄ ∨/b←⍵∊⎕null:{(b/a)←⍵⋄a}⊂'null' ⋄ ⍵}¨data
json←⎕json ⊂ 4 (data col)

⍝ KSL 1.0.2
```

In [16]:
]multiline
cons_daily←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''day'',time),''%Y-%m-%d'') as date,
  round(sum(consumption),2) as cons_daily
FROM elec
WHERE year(time)>2021
GROUP BY date_trunc(''day'',time)
ORDER BY date
``` ⋄ 100↑cons_daily

[{"date":"2022-01-01","cons_daily":144.29},{"date":"2022-01-02","cons_daily":133},{"date":"2022-01-0

In [17]:
]multiline
cons_monthly←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''month'',time),''%Y-%m-%d'')as month,
  round(sum(consumption),2) as cons_monthly
FROM elec
WHERE year(time)>2021
GROUP BY date_trunc(''month'',time)
ORDER BY month
``` ⋄ 100↑cons_monthly

[{"month":"2022-01-01","cons_monthly":3906.38},{"month":"2022-02-01","cons_monthly":3078.02},{"month

In [18]:
]multiline
price_daily←duckdb.toJson duckdb.query _con ```
SELECT
  strftime(date_trunc(''day'',time),''%Y-%m-%d'') as date,
  min(price) as min,
  max(price) as max
FROM elec
WHERE year(time)>2021
GROUP BY date_trunc(''day'',time)
ORDER BY date
``` ⋄ 100↑price_daily

[{"date":"2022-01-01","min":4.671080000000001,"max":18.59628},{"date":"2022-01-02","min":2.59036,"ma

In [19]:
cons_daily ⎕nput 'data/cons_daily.json' 1
cons_monthly ⎕nput 'data/cons_monthly.json' 1
price_daily ⎕nput 'data/price_daily.json' 1

In [20]:
]vegalite
'plots/layerplot_url.vl'layerPlot ('data/cons_daily.json' 'data/cons_monthly.json')

In [21]:
]vegalite
'plots/lineplot_url.vl'linePlot ('data/price_daily.json')

In [22]:
('plots/lineplot.vl'linePlot price_daily) ⎕nput 'data/test.vl' 1

In [23]:
⎕SH 'vl2pdf data/test.vl testi.pdf'
⎕SH 'vl2png data/test.vl testi.png'
⎕SH 'vl2svg data/test.vl testi.svg'

In [24]:
_←duckdb.disconnect _con
_←duckdb.close _db 